#Training Code

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Sem Eval Task 11 Group 7/Train v2.zip" -d "/content/train"

Archive:  /content/drive/MyDrive/Sem Eval Task 11 Group 7/Train v2.zip
   creating: /content/train/Train v2/
   creating: /content/train/Train v2/natural_language_inference/
   creating: /content/train/Train v2/natural_language_inference/0/
  inflating: /content/train/Train v2/natural_language_inference/0/1606.01549v3-Grobid-out.txt  
  inflating: /content/train/Train v2/natural_language_inference/0/1606.01549v3-Stanza-out.txt  
  inflating: /content/train/Train v2/natural_language_inference/0/1606.01549v3.pdf  
  inflating: /content/train/Train v2/natural_language_inference/0/entities.txt  
   creating: /content/train/Train v2/natural_language_inference/0/info-units/
  inflating: /content/train/Train v2/natural_language_inference/0/info-units/ablation-analysis.json  
  inflating: /content/train/Train v2/natural_language_inference/0/info-units/code.json  
  inflating: /content/train/Train v2/natural_language_inference/0/info-units/model.json  
  inflating: /content/train/Train v2/natur

In [ ]:
!unzip "/content/drive/MyDrive/Sem Eval Task 11 Group 7/Dev v2.zip" -d "/content/valid"

Archive:  /content/drive/MyDrive/Sem Eval Task 11 Group 7/Dev v2.zip
   creating: /content/valid/Dev v2/
   creating: /content/valid/Dev v2/machine-translation/
   creating: /content/valid/Dev v2/machine-translation/0/
  inflating: /content/valid/Dev v2/machine-translation/0/1606.04199v3-Grobid-out.txt  
  inflating: /content/valid/Dev v2/machine-translation/0/1606.04199v3-Stanza-out.txt  
  inflating: /content/valid/Dev v2/machine-translation/0/1606.04199v3.pdf  
  inflating: /content/valid/Dev v2/machine-translation/0/entities.txt  
   creating: /content/valid/Dev v2/machine-translation/0/info-units/
  inflating: /content/valid/Dev v2/machine-translation/0/info-units/experimental-setup.json  
  inflating: /content/valid/Dev v2/machine-translation/0/info-units/model.json  
  inflating: /content/valid/Dev v2/machine-translation/0/info-units/research-problem.json  
  inflating: /content/valid/Dev v2/machine-translation/0/info-units/results.json  
  inflating: /content/valid/Dev v2/machi

In [ ]:
# Training dataset reading
input_dir = "/content/train/Train v2/"
import os
import torch
list_of_folders = ["query_wellformedness", "passage_re-ranking", "part-of-speech_tagging", 
         "sentence_compression", "sentiment_analysis", "temporal_information_extraction", 
         "phrase_grounding", "text_generation", "text-to-speech_synthesis", 
         "smile_recognition", "topic_models", "question_generation", 
         "relation_extraction", "paraphrase_generation", "question_similarity", 
         "question_answering", "sentence_classification", "prosody_prediction", 
         "semantic_role_labeling", "text_summarization", "semantic_parsing", 
         "sarcasm_detection", "natural_language_inference", "negation_scope_resolution"]
input_stanza_list = []
input_sent_num_list = []
input_entity_list = []
file_name_list = []
total_phrases_truth = 0
for fls in list_of_folders:
  count=0
  for i in os.listdir(input_dir + fls + '/'):
    count=count+1
    for files in os.listdir(input_dir + fls + '/' + str(i)):
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        stanza_lines = (stanza_file.read()).lower()
        stanza_lines_list = list(filter(None,stanza_lines.splitlines())) # filter empty strings and split into lines
        input_stanza_list.append(stanza_lines_list)
      if files.endswith("sentences.txt"):
        sentence_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        sentence_num_list = list(filter(None,(sentence_file.read().lower()).splitlines())) # filter empty strings and split into lines
        input_sent_num_list.append(list(map(int, sentence_num_list)))
      if files.endswith("entities.txt"):
        entities_file = open(input_dir + fls + '/' + str(i) + '/' + files, "r")
        entities_list = list(filter(None,(entities_file.read().lower()).splitlines())) # filter empty strings and split into lines
        input_entity_list.append(entities_list)
        total_phrases_truth = total_phrases_truth + len(entities_list)
    file_name_list.append(fls + '/' + str(i))
  print("completed",fls,total_phrases_truth, count)


completed query_wellformedness 55 1
completed passage_re-ranking 270 2
completed part-of-speech_tagging 1084 8
completed sentence_compression 1513 4
completed sentiment_analysis 9006 52
completed temporal_information_extraction 9161 2
completed phrase_grounding 9334 1
completed text_generation 10049 6
completed text-to-speech_synthesis 10375 3
completed smile_recognition 10461 1
completed topic_models 10535 1
completed question_generation 10668 2
completed relation_extraction 12596 14
completed paraphrase_generation 12891 2
completed question_similarity 12975 1
completed question_answering 14045 6
completed sentence_classification 14554 3
completed prosody_prediction 14729 1
completed semantic_role_labeling 15275 5
completed text_summarization 17059 15
completed semantic_parsing 17337 3
completed sarcasm_detection 17560 2
completed natural_language_inference 30291 101
completed negation_scope_resolution 30445 1


In [ ]:
# Validation dataset reading
val_input_dir = "/content/valid/Dev v2/"
val_list_of_folders = ["machine-translation", "named-entity-recognition", "question-answering",
         "relation-classification", "text-classification"]
val_input_stanza_list = []
val_input_sent_num_list = []
val_input_entity_list = []
val_file_name_list = []
val_total_phrases_truth = 0
for fls in val_list_of_folders:
  count=0
  for i in os.listdir(val_input_dir + fls + '/'):
    count=count+1
    for files in os.listdir(val_input_dir + fls + '/' + str(i)):
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(val_input_dir + fls + '/' + str(i) + '/' + files, "r")
        stanza_lines = stanza_file.read().lower()
        stanza_lines_list = list(filter(None,stanza_lines.splitlines())) # filter empty strings and split into lines
        val_input_stanza_list.append(stanza_lines_list)
      if files.endswith("sentences.txt"):
        sentence_file = open(val_input_dir + fls + '/' + str(i) + '/' + files, "r")
        sentence_num_list = list(filter(None,(sentence_file.read().lower()).splitlines())) # filter empty strings and split into lines
        val_input_sent_num_list.append(list(map(int, sentence_num_list)))
      if files.endswith("entities.txt"):
        entities_file = open(val_input_dir + fls + '/' + str(i) + '/' + files, "r")
        entities_list = list(filter(None,(entities_file.read().lower()).splitlines())) # filter empty strings and split into lines
        val_input_entity_list.append(entities_list)
        val_total_phrases_truth = val_total_phrases_truth + len(entities_list)
    val_file_name_list.append(fls + '/' + str(i))
  print("completed",fls,val_total_phrases_truth,count)

completed machine-translation 915 10
completed named-entity-recognition 1707 10
completed question-answering 2738 10
completed relation-classification 3728 10
completed text-classification 4777 10


In [ ]:
def make_substring(n): # replace phrases with labels
  if n==0:
    return ''
  elif n ==1:
    return 'U'
  elif n==2:
    return 'B L'
  else:
    t1 = 'I '*(n-2)
    return 'B '+t1+'L'

In [ ]:
# train function
import copy
task2_in = [] #input sentences of examples
task2_label = []# replaced the input by labels for phrases
for i in range(len(input_stanza_list)): #process each file for labels
  entity_list = [j.split('\t') for j in input_entity_list[i]] # split the entities line
  entity_list.sort(key=lambda x: (int(x[0]),int(x[1]))) # arrange in ascending order w.r.t sentence, char number
  sent_num_list = copy.deepcopy(input_sent_num_list[i]) # copy of the sentences list
  sent_num_list.sort()                          # sorting the sentences list
  sent_list = []  # list containg sentence strings
  
  for x in sent_num_list: 
    sent_list.append(input_stanza_list[i][x-1])
  
  sent_dict_list = dict(zip(sent_num_list,sent_list)) # dictionary of sentence number and strings
  for n ,ind_s ,ind_e, ph in entity_list: # entity list to label formation
    
    if int(n) in sent_num_list:
      sent_dict_list[int(n)] = sent_dict_list[int(n)].replace(ph,make_substring(len(ph.split())),1)
      
  sent_label_list = list(sent_dict_list.values())
  task2_in.append(sent_list)
  task2_label.append(sent_label_list)

In [ ]:
# valid function
import copy
val_task2_in = [] #input sentences of examples
val_task2_label = []# replaced the input by labels for phrases
for i in range(len(val_input_stanza_list)): #process each file for labels
  val_entity_list = [j.split('\t') for j in val_input_entity_list[i]] # split the entities line
  val_entity_list.sort(key=lambda x: (int(x[0]),int(x[1]))) # arrange in ascending order w.r.t sentence, char number
  val_sent_num_list = copy.deepcopy(val_input_sent_num_list[i]) # copy of the sentences list
  val_sent_num_list.sort()                          # sorting the sentences list
  val_sent_list = []  # list containg sentence strings
  #print(sentence_num_list)
  for x in val_sent_num_list: 
    val_sent_list.append(val_input_stanza_list[i][x-1])
  #print(c,len(sent_list),len(c))
  #print(sent_list[0])
  val_sent_dict_list = dict(zip(val_sent_num_list,val_sent_list)) # dictionary of sentence number and strings
  for n ,ind_s ,ind_e, ph in val_entity_list: # entity list to label formation
    if int(n) in val_sent_num_list:
      val_sent_dict_list[int(n)] = val_sent_dict_list[int(n)].replace(ph,make_substring(len(ph.split())),1)
      
  val_sent_label_list = list(val_sent_dict_list.values())
  val_task2_in.append(val_sent_list)
  val_task2_label.append(val_sent_label_list)

In [ ]:
# train dataset replacing all unnecessary tokens with 'O' token 
for i,out in enumerate(task2_label): 
  for j,line in enumerate(out):
    for k,tok in enumerate(line.split()):
      if tok not in ['B','I','L','U']:
        task2_label[i][j]=task2_label[i][j].replace(tok,'O',1)

In [ ]:
# valid dataset replacing all unnecessary tokens with 'O' token 
for i,out in enumerate(val_task2_label): 
  for j,line in enumerate(out):
    for k,tok in enumerate(line.split()):
      if tok not in ['B','I','L','U']:
        val_task2_label[i][j]=val_task2_label[i][j].replace(tok,'O',1)

In [ ]:
# Helper Functions
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import time
torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq):
    for count,i in enumerate(seq):
       temp = tokenizer.tokenize(i) 
       if(len(temp)>1):
         seq[count] = temp[0]           
    #idxs = [to_ix[w] for w in seq if w in to_ix.keys()]
    sentences = " ".join(seq)
    inputs = tokenizer(sentences, return_tensors="pt")
    return inputs


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [ ]:
#Defining the Model

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        #Pretrianed SciBert downloaded from allenai
        self.modell = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        
        outputs = self.modell(**sentence, output_hidden_states = True)
        scibert_out = ((outputs[2][12])[0]).view(len(sentence["input_ids"][0]), 1, -1)
        self.hidden = self.init_hidden()
        lstm_out, self.hidden = self.lstm(scibert_out, self.hidden)
        lstm_out = lstm_out.view(len(sentence["input_ids"][0]), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = [] # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):

        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [ ]:
#Model hyperparameters and Word to index dictionary defined
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 768
HIDDEN_DIM = 200

In [ ]:
word_to_ix = {'<UNK>': 0}
for i,file in enumerate(task2_in):
  for j,sent in enumerate(file):
    for k,tok in enumerate(sent.split()):
       if tok not in word_to_ix:
         word_to_ix[tok] = len(word_to_ix)

In [ ]:
tag_to_ix = {"B": 0, "I": 1, "L": 2, "U": 3, "O": 4, START_TAG: 5, STOP_TAG: 6}
ix_to_tag = {v: k for k, v in tag_to_ix.items()}

In [ ]:
!pip install transformers==3.5

from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

     |████████████████████████████████| 1.3MB 15.0MB/s 
     |████████████████████████████████| 2.9MB 46.3MB/s 
     |████████████████████████████████| 1.1MB 51.4MB/s 
     |████████████████████████████████| 890kB 49.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=fbc2f72d9e00b194c86811080f12b8f8d23919fa6130810a830ae938b6bdcec5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
# Model initialized and Seeds defined
import random
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = BiLSTM_CRF(12345, tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
model.cuda()
seed_val = 66
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
def calc_eval(listp, listg):
  correct = 0
  total_p = 0
  for i in zip(listp, listg):
    flag = 0
    for j in zip(i[0], i[1]):
      if j[0]=='U' or j[0]=='B':
        total_p += 1
      if flag==0:
        if j[0]=='U' and j[1]=='U':
          correct += 1
        if j[0]=='B' and j[1]=='B':
          flag = 1
      else:
        if j[0]=='L' and j[1]=='L':
          correct += 1
          flag = 0
        elif j[0]==j[1]:
          continue
        else:
          flag = 0
  return correct,total_p

In [ ]:
def training_eval():
  task2_out_label = []
  for i,file in enumerate(task2_in):
    output = []
    for j,sent in enumerate(file):
      with torch.no_grad():
        precheck_sent = prepare_sequence(task2_in[i][j].split()).to(device)
        sent_out_label = model(precheck_sent)[1]
        sent_str_label = [ix_to_tag[t] for t in sent_out_label]
        #print(sent_str_label)
        #break
        output.append(sent_str_label)
    task2_out_label.append(output)
    #break
  print(len(task2_out_label))
  true_pos = 0
  total_ph_pred = 0
  for i,file in enumerate(task2_label):
    file_true_pos, file_total_ph_pred = calc_eval(task2_out_label[i],[["O"] + s.split() + ["O"] for s in task2_label[i]])
    true_pos = true_pos + file_true_pos
    total_ph_pred = total_ph_pred + file_total_ph_pred
    if i%20==0:
      print("Each example",i,file_true_pos,file_total_ph_pred)
  print(true_pos,total_ph_pred,total_phrases_truth)
  precision = 0
  recall = 0
  F1score=  0
  if(total_ph_pred!=0):
    precision = true_pos/total_ph_pred
  if(total_phrases_truth!=0):
    recall = true_pos/total_phrases_truth
  if((precision + recall)!=0):  
    F1score = 2 * precision*recall/(precision+recall)
  print("Precision is {} and recall is {} and F1 Score is {}".format(precision,recall,F1score))

In [ ]:
def validation_eval():
  val_task2_out_label = []
  for i,file in enumerate(val_task2_in):
    output = []
    for j,sent in enumerate(file):
      with torch.no_grad():
        precheck_sent = prepare_sequence(val_task2_in[i][j].split()).to(device)
        sent_out_label = model(precheck_sent)[1]
        sent_str_label = [ix_to_tag[t] for t in sent_out_label]
        #print(sent_str_label)
        #break
        output.append(sent_str_label)
    val_task2_out_label.append(output)
    #break
  print(len(val_task2_out_label))
  val_true_pos = 0
  val_total_ph_pred = 0
  for i,file in enumerate(val_task2_label):
    val_file_true_pos, val_file_total_ph_pred = calc_eval(val_task2_out_label[i],[["O"] + s.split() + ["O"] for s in val_task2_label[i]])
    val_true_pos = val_true_pos + val_file_true_pos
    val_total_ph_pred = val_total_ph_pred + val_file_total_ph_pred
    #if i%20==0:
    print("Each example",i,"true pos",val_file_true_pos,"phrase in pred",val_file_total_ph_pred)
  print(val_true_pos,val_total_ph_pred,val_total_phrases_truth)
  val_precision = 0
  val_recall = 0
  val_F1score=  0
  if(val_total_ph_pred!=0):
    val_precision = val_true_pos/val_total_ph_pred
  if(val_total_phrases_truth!=0):
    val_recall = val_true_pos/val_total_phrases_truth
  if((val_precision + val_recall)!=0):  
    val_F1score = 2 * val_precision*val_recall/(val_precision+val_recall)
  print("Precision is {} and recall is {} and F1 Score is {}".format(val_precision,val_recall,val_F1score))


# Training 

In [ ]:
#The MAIN TRAINING CELL

#Check predictions before training
with torch.no_grad():
    #print(task2_in[0][0].split())
    precheck_sent = prepare_sequence(task2_in[0][0].split()).to(device)
    #print(precheck_sent)
    precheck_tags = torch.tensor([4] + [tag_to_ix[t] for t in task2_label[0][0].split()] + [4], dtype=torch.long).to(device)
    print(precheck_tags)
    print(model(precheck_sent),'\n',precheck_tags)
    print("checkpoint passed")


for epoch in range(50):  #  or 300 epochs
    start = time.time()
    model.train()
    for i,file in enumerate(task2_in):
      for j,sent in enumerate(file):
        
        model.zero_grad()

        sentence_in = prepare_sequence(sent.split()).to(device)
    
        targets = torch.tensor([4] + [tag_to_ix[t] for t in task2_label[i][j].split()] + [4], dtype=torch.long).to(device)
        
        loss = model.neg_log_likelihood(sentence_in, targets)

        loss.backward()

        optimizer.step()
    end = time.time()

    #Saving the Model, note that it will be very heavy  
    """
    if epoch%1 == 0: 
      torch.save(model,"/content/drive/My Drive/train/ULTIMATEEEEEEEEE_finality_GPU_scibert_LSTM_adam_validation_train_version1_mod" + str(epoch) + ".pt")
    """

    print("The epoch completed is",epoch, "and time", end-start)

    #Validation Loss computation
    if epoch%1 ==0:
      model.eval()
      training_eval()
      validation_eval()
      val_total_loss = 0.
      with torch.no_grad():
        for k, val_file in enumerate(val_task2_in):
          for l, val_sent in enumerate(val_file):
            val_sentence_in = prepare_sequence(val_sent.split()).to(device)
            val_targets = torch.tensor([4] + [tag_to_ix[t] for t in val_task2_label[k][l].split()] + [4], dtype=torch.long).to(device)
            loss = model.neg_log_likelihood(val_sentence_in, val_targets)
            val_total_loss += loss.item()
      print("validation loss after epoch",epoch," is", val_total_loss)
      

# Simple check
with torch.no_grad():
    precheck_sent = prepare_sequence(task2_in[0][0].split()).to(device)
    print(model(precheck_sent))

In [ ]:
#Model Evaluation
model.eval()

#Train Metrics
training_eval()

#Validation Metrics
validation_eval()

237
Each example 0 37 50
Each example 20 197 226
Each example 40 117 141
Each example 60 73 94
Each example 80 55 106
Each example 100 125 142
Each example 120 103 154
Each example 140 60 67
Each example 160 121 143
Each example 180 177 196
Each example 200 134 179
Each example 220 143 164
25989 31053 30445
Precision is 0.8369239686986765 and recall is 0.8536377073411069 and F1 Score is 0.8451982178282219
50
Each example 0 true pos 18 phrase in pred 71
Each example 1 true pos 48 phrase in pred 114
Each example 2 true pos 92 phrase in pred 233
Each example 3 true pos 20 phrase in pred 56
Each example 4 true pos 55 phrase in pred 105
Each example 5 true pos 99 phrase in pred 209
Each example 6 true pos 74 phrase in pred 132
Each example 7 true pos 37 phrase in pred 80
Each example 8 true pos 80 phrase in pred 148
Each example 9 true pos 73 phrase in pred 182
Each example 10 true pos 51 phrase in pred 95
Each example 11 true pos 91 phrase in pred 173
Each example 12 true pos 50 phrase in 

#Evaluation Script (StreamLined)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Helper Functions
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import time
torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq):
    for count,i in enumerate(seq):
       temp = tokenizer.tokenize(i) 
       if(len(temp)>1):
         seq[count] = temp[0]           
    #idxs = [to_ix[w] for w in seq if w in to_ix.keys()]
    sentences = " ".join(seq)
    inputs = tokenizer(sentences, return_tensors="pt")
    return inputs


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [ ]:
#Defining the Model

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        #Pretrianed SciBert downloaded from allenai
        self.modell = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        
        outputs = self.modell(**sentence, output_hidden_states = True)
        scibert_out = ((outputs[2][12])[0]).view(len(sentence["input_ids"][0]), 1, -1)
        self.hidden = self.init_hidden()
        lstm_out, self.hidden = self.lstm(scibert_out, self.hidden)
        lstm_out = lstm_out.view(len(sentence["input_ids"][0]), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = [] # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):

        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [ ]:
#Model hyperparameters and Word to index dictionary defined
START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 768
HIDDEN_DIM = 200

In [ ]:
tag_to_ix = {"B": 0, "I": 1, "L": 2, "U": 3, "O": 4, START_TAG: 5, STOP_TAG: 6}
ix_to_tag = {v: k for k, v in tag_to_ix.items()}

In [ ]:
!pip install transformers==3.5

from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")

     |████████████████████████████████| 1.3MB 7.7MB/s 
     |████████████████████████████████| 2.9MB 14.4MB/s 
     |████████████████████████████████| 890kB 35.4MB/s 
     |████████████████████████████████| 1.1MB 36.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=c1d56f3259874885ee7cb673b5a48628c456ad316b78a947686d096bc448bd5a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
# Model initialized and Seeds defined
import random
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = BiLSTM_CRF(12345, tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
model.cuda()
seed_val = 66
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
model = torch.load("/content/drive/MyDrive/CS779_All_Subtasks/Final_sub_task_2_scibert_LSTM_1.pt")
model.eval()

BiLSTM_CRF(
  (lstm): LSTM(768, 100, bidirectional=True)
  (hidden2tag): Linear(in_features=200, out_features=7, bias=True)
  (modell): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=76

In [ ]:
def calc_eval(listp, listg):
  correct = 0
  total_p = 0
  for i in zip(listp, listg):
    flag = 0
    for j in zip(i[0], i[1]):
      if j[0]=='U' or j[0]=='B':
        total_p += 1
      if flag==0:
        if j[0]=='U' and j[1]=='U':
          correct += 1
        if j[0]=='B' and j[1]=='B':
          flag = 1
      else:
        if j[0]=='L' and j[1]=='L':
          correct += 1
          flag = 0
        elif j[0]==j[1]:
          continue
        else:
          flag = 0
  return correct,total_p

In [ ]:
def make_substring(n): # replace phrases with labels
  if n==0:
    return ''
  elif n ==1:
    return 'U'
  elif n==2:
    return 'B L'
  else:
    t1 = 'I '*(n-2)
    return 'B '+t1+'L'

In [ ]:
#output_dir = "/content/drive/MyDrive/result_cs779/"

In [ ]:
# Test dataset reading
import os
test_input_dir = "/content/drive/MyDrive/submission8/"
# test_list_of_folders = ["machine-translation", "named-entity-recognition", "question-answering",
#          "relation-classification", "text-classification"]
#test_list_of_folders = ["entity-linking", "face-alignment", "face-detection", "natural-language-inference"]
test_list_of_folders = ["constituency_parsing","coreference_resolution",
                   "data-to-text_generation","dependency_parsing",
                   "document_classification","entity_linking",
                   "face_alignment","face_detection", "hypernym_discovery",
                   "natural_language_inference"]
test_input_stanza_list = []
test_input_sent_num_list = []
test_input_entity_list = []
test_file_name_list = []
test_total_phrases_truth = 0
Capital_test_input_stanza_list = []
for fls in test_list_of_folders:
  count=0
  for i in os.listdir(test_input_dir + fls + '/'):
    count=count+1
    for files in os.listdir(test_input_dir + fls + '/' + str(i)):
      if files.endswith("Stanza-out.txt"):
        stanza_file = open(test_input_dir + fls + '/' + str(i) + '/' + files, "r")
        print(test_input_dir + fls + '/' + str(i))
        Capital_stanza_lines = stanza_file.read()
        Capital_stanza_lines_list = list(filter(None,Capital_stanza_lines.splitlines())) # filter empty strings and split into lines
        Capital_test_input_stanza_list.append(Capital_stanza_lines_list)

        stanza_lines = Capital_stanza_lines.lower()
        stanza_lines_list = list(filter(None,stanza_lines.splitlines())) # filter empty strings and split into lines
        test_input_stanza_list.append(stanza_lines_list)
      if files.endswith("sentences.txt"):
        sentence_file = open(test_input_dir + fls + '/' + str(i) + '/' + 'sentences.txt', "r")
        sentence_num_list = list(filter(None,(sentence_file.read().lower()).splitlines())) # filter empty strings and split into lines
        test_input_sent_num_list.append(list(map(int, sentence_num_list)))
      # if files.endswith("entities.txt"):
      #   entities_file = open(test_input_dir + fls + '/' + str(i) + '/' + files, "r")
      #   entities_list = list(filter(None,(entities_file.read().lower()).splitlines())) # filter empty strings and split into lines
      #   test_input_entity_list.append(entities_list)
      #   test_total_phrases_truth = test_total_phrases_truth + len(entities_list)
    test_file_name_list.append(fls + '/' + str(i))
  print("completed",fls,count)
##########Check test data
print("Test no. of examples for each stanza,sentences and entities")
print(len(test_input_sent_num_list),len(test_file_name_list))
#print(len(test_input_entity_list),len(test_input_sent_num_list),len(test_input_stanza_list),len(test_file_name_list))
#test_input_sent_num_list = [[int(s) for s in sublist] for sublist in test_input_sent_num_list] # convert sentence list string to integer
#test_input_sent_num_list = [list(set(x)) for x in test_input_sent_num_list]
print("print one example to show all files")
print(len(test_input_sent_num_list[0]),test_file_name_list[0])
#print(len(test_input_entity_list[0]),len(test_input_sent_num_list[0]),len(test_input_stanza_list[0]),test_file_name_list[0])

/content/drive/MyDrive/submission8/constituency_parsing/6
/content/drive/MyDrive/submission8/constituency_parsing/5
/content/drive/MyDrive/submission8/constituency_parsing/8
/content/drive/MyDrive/submission8/constituency_parsing/7
/content/drive/MyDrive/submission8/constituency_parsing/0
/content/drive/MyDrive/submission8/constituency_parsing/4
/content/drive/MyDrive/submission8/constituency_parsing/3
/content/drive/MyDrive/submission8/constituency_parsing/2
/content/drive/MyDrive/submission8/constituency_parsing/1
completed constituency_parsing 9
/content/drive/MyDrive/submission8/coreference_resolution/9
/content/drive/MyDrive/submission8/coreference_resolution/6
/content/drive/MyDrive/submission8/coreference_resolution/8
/content/drive/MyDrive/submission8/coreference_resolution/7
/content/drive/MyDrive/submission8/coreference_resolution/4
/content/drive/MyDrive/submission8/coreference_resolution/2
/content/drive/MyDrive/submission8/coreference_resolution/5
/content/drive/MyDrive/su

In [ ]:
#########test function
import copy
test_task2_in = [] #input sentences of examples
test_task2_label = []# replaced the input by labels for phrases
Capital_test_task2_in = []
for i in range(len(test_input_sent_num_list)): #process each file for labels
  #test_entity_list = [j.split('\t') for j in test_input_entity_list[i]] # split the entities line
  #test_entity_list.sort(key=lambda x: (int(x[0]),int(x[1]))) # arrange in ascending order w.r.t sentence, char number
  test_sent_num_list = copy.deepcopy(test_input_sent_num_list[i]) # copy of the sentences list
  test_sent_num_list.sort()                          # sorting the sentences list
  test_sent_list = []  # list containg sentence strings
  Capital_test_sent_list = []
  #print(sentence_num_list)
  for x in test_sent_num_list: 
    test_sent_list.append(test_input_stanza_list[i][x-1])
    Capital_test_sent_list.append(Capital_test_input_stanza_list[i][x-1])
  #print(c,len(sent_list),len(c))
  #print(sent_list[0])
  test_sent_dict_list = dict(zip(test_sent_num_list,test_sent_list)) # dictionary of sentence number and strings
  # for n ,ind_s ,ind_e, ph in test_entity_list: # entity list to label formation
  #   if int(n) in test_sent_num_list:
  #     test_sent_dict_list[int(n)] = test_sent_dict_list[int(n)].replace(ph,make_substring(len(ph.split())),1)
      
  test_sent_label_list = list(test_sent_dict_list.values())
  test_task2_in.append(test_sent_list)
  Capital_test_task2_in.append(Capital_test_sent_list)
  test_task2_label.append(copy.deepcopy(test_sent_label_list))


In [ ]:
# Check the test label processing
for i in range(len(test_task2_in)):
  if len(test_task2_in[i]) != len(test_task2_label[i]):
    print(len(test_task2_in[i]),len(test_task2_label[i]),i)
# Flag checker to check whether all sizes are correct or not
for i,file in enumerate(test_task2_in):
    for j,sent in enumerate(file):
      temp1 = sent.split()
      temp2 = test_task2_label[i][j].split()
      if len(temp1)!=len(temp2):
        print(len(temp1), len(temp2), i,j,test_file_name_list[i])
        print(sent,temp1,temp2)

In [ ]:
# Test dataset replacing all unnecessary tokens with 'O' token 
for i,out in enumerate(test_task2_label): 
  for j,line in enumerate(out):
    for k,tok in enumerate(line.split()):
      if tok not in ['B','I','L','U']:
        test_task2_label[i][j]=test_task2_label[i][j].replace(tok,'O',1)

In [ ]:
#This is where we make a dictionary that can map sentence to its corresponding number in stanza_out.txt 

import copy
list_of_dict_for_number_to_sentence = []
for i in range(len(test_input_stanza_list)): #process each file for labels
  # test_entity_list = [j.split('\t') for j in test_input_entity_list[i]] # split the entities line
  # test_entity_list.sort(key=lambda x: (int(x[0]),int(x[1]))) # arrange in ascending order w.r.t sentence, char number
  test_sent_num_list = copy.deepcopy(test_input_sent_num_list[i]) # copy of the sentences list
  test_sent_num_list.sort()                          # sorting the sentences list
  test_sent_list = []  # list containg sentence strings

  for x in test_sent_num_list: 
    test_sent_list.append(test_input_stanza_list[i][x-1])
  test_sent_dict_list = dict(zip(test_sent_num_list,test_sent_list)) # dictionary of sentence number and strings
  list_of_dict_for_number_to_sentence.append(test_sent_dict_list)  

#This is the mapper list of dictionaries
list_of_dict_for_sentence_to_number = [dict((v,k) for k,v in a.items()) for a in list_of_dict_for_number_to_sentence]

In [ ]:
# #####test label evaluate
test_task2_out_label = []
for i,file in enumerate(test_task2_in):
  output = []
  for j,sent in enumerate(file):
    with torch.no_grad():
      precheck_sent = prepare_sequence(test_task2_in[i][j].split()).to(device)
      sent_out_label = model(precheck_sent)[1]
      sent_str_label = [ix_to_tag[t] for t in sent_out_label]
      output.append(sent_str_label)
  test_task2_out_label.append(output)
print(len(test_task2_out_label))
test_true_pos = 0
test_total_ph_pred = 0
for i,file in enumerate(test_task2_label):
  test_file_true_pos, test_file_total_ph_pred = calc_eval(test_task2_out_label[i],[["O"] + s.split() + ["O"] for s in test_task2_label[i]])
  test_true_pos = test_true_pos + test_file_true_pos
  test_total_ph_pred = test_total_ph_pred + test_file_total_ph_pred
  if i%5==0:
    print("Each example",i,"true pos",test_file_true_pos,"phrase in pred",test_file_total_ph_pred)
print(test_true_pos,test_total_ph_pred,test_total_phrases_truth)


# test_precision = 0
# test_recall = 0
# test_F1score=  0
# if(test_total_ph_pred!=0):
#   test_precision = test_true_pos/test_total_ph_pred
# if(test_total_phrases_truth!=0):
#   test_recall = test_true_pos/test_total_phrases_truth
# if((test_precision + test_recall)!=0):  
#   test_F1score = 2 * test_precision*test_recall/(test_precision+test_recall)
# print("Precision is {} and recall is {} and F1 Score is {}".format(test_precision,test_recall,test_F1score))


155
Each example 0 true pos 0 phrase in pred 81
Each example 5 true pos 0 phrase in pred 151
Each example 10 true pos 0 phrase in pred 128
Each example 15 true pos 0 phrase in pred 174
Each example 20 true pos 0 phrase in pred 116
Each example 25 true pos 0 phrase in pred 197
Each example 30 true pos 0 phrase in pred 119
Each example 35 true pos 0 phrase in pred 113
Each example 40 true pos 0 phrase in pred 50
Each example 45 true pos 0 phrase in pred 135
Each example 50 true pos 0 phrase in pred 270
Each example 55 true pos 0 phrase in pred 112
Each example 60 true pos 0 phrase in pred 180
Each example 65 true pos 0 phrase in pred 77
Each example 70 true pos 0 phrase in pred 153
Each example 75 true pos 0 phrase in pred 276
Each example 80 true pos 0 phrase in pred 179
Each example 85 true pos 0 phrase in pred 134
Each example 90 true pos 0 phrase in pred 81
Each example 95 true pos 0 phrase in pred 213
Each example 100 true pos 0 phrase in pred 237
Each example 105 true pos 0 phrase 

In [ ]:
# %rm -r "/content/machine-translation"
# %rm -r "/content/named-entity-recognition"
# %rm -r "/content/question-answering"
# %rm -r "/content/relation-classification"
# %rm -r "/content/text-classification"

In [ ]:
# %mkdir "/content/machine-translation"
# %mkdir "/content/named-entity-recognition"
# %mkdir "/content/question-answering"
# %mkdir "/content/relation-classification"
# %mkdir "/content/text-classification"

In [ ]:
#### Undo the third task inference (Delete all the triples folder in the test data)
import os
import shutil
#####Test model
#test_input_dir = "/content/drive/My Drive/Dev_v2/"
test_input_dir = "/content/drive/MyDrive/submission8/"
# test_list_of_folders = ["machine-translation", "named-entity-recognition", "question-answering",
#          "relation-classification", "text-classification"]
#test_list_of_folders = ["entity-linking", "face-alignment", "face-detection", "natural-language-inference"]
test_list_of_folders = ["constituency_parsing","coreference_resolution",
                   "data-to-text_generation","dependency_parsing",
                   "document_classification","entity_linking",
                   "face_alignment","face_detection", "hypernym_discovery",
                   "natural_language_inference"]
for fls in test_list_of_folders:
  for i in os.listdir(test_input_dir + fls + '/'):
    for ii in os.listdir(test_input_dir + fls + '/' + str(i) + "/"):
      if(ii=="entities.txt"):
       os.remove(test_input_dir + fls + '/' + str(i) + "/" + "entities.txt")

In [ ]:
import copy
print(len(test_task2_in))
print(len(test_task2_in[0]))
print(test_task2_in[0])
print(len(Capital_test_task2_in))
print(len(Capital_test_task2_in[0]))
print(Capital_test_task2_in[0])
print(test_file_name_list[0])
print(len(test_task2_out_label))
print(len(test_task2_out_label[0]))
print((test_task2_out_label[0]))
print((test_task2_in[0][2]))
print(len(test_task2_out_label[0][2]))
print(len(test_task2_in[0][2].split()))

output_dir = "/content/drive/MyDrive/submission8/"

phrase_storer = []
f = open("demofile3.txt", "w")
for i,file in enumerate(test_task2_in):
  
  print(test_file_name_list[i])
  # filenametemp = "/content/" + test_file_name_list[i]
  # %mkdir $filenametemp
  f1 = open(output_dir + test_file_name_list[i] + "/entities.txt", "w")

  for j,sent in enumerate(file):

    biluo_list = (test_task2_out_label[i][j])[1:-1]
    respective_sentence = Capital_test_task2_in[i][j].split()
    sentence_number = (list_of_dict_for_sentence_to_number[i])[test_task2_in[i][j]]

    if(len(respective_sentence) != len(biluo_list)):
      print("Length mismatch in the sentence and BILUO sequence")
      continue

    temp_phrase_storer = []
    temp_phrase = []
    count_of_words_in_sentence = 0
    for k in zip(biluo_list,respective_sentence):

      if (k[0]=="U"):
        temp_phrase_storer = temp_phrase_storer + [k[1]]

        start_of_word = 0
        if(count_of_words_in_sentence == 0):
          start_of_word = 0
        else:  
          start_of_word = len((" ".join(respective_sentence[0:count_of_words_in_sentence])).strip() + " ") 

        end_of_word = start_of_word + len(k[1].strip())

        f.write(str(sentence_number) + "\t" +  str(start_of_word) + "\t" + str(end_of_word) + "\t" + k[1].strip() + "\n")
        f1.write(str(sentence_number) + "\t" +  str(start_of_word) + "\t" + str(end_of_word) + "\t" + k[1].strip() + "\n")

      elif (k[0]=="B"):
        temp_phrase = temp_phrase + [k[1]] 

      elif (k[0]=="I"):
        temp_phrase = temp_phrase + [" ", k[1]]

      elif (k[0]=="L"):
        temp_phrase = temp_phrase + [" ", k[1]]

        end_of_words = len((" ".join(respective_sentence[0:count_of_words_in_sentence])).strip() + " ") + len(respective_sentence[count_of_words_in_sentence].strip())
        start_of_words = end_of_words - len(("".join(temp_phrase)).strip())

        f.write(str(sentence_number) + "\t" + str(start_of_words) + "\t" + str(end_of_words) + "\t" + ("".join(temp_phrase)).strip() + "\n")
        f1.write(str(sentence_number) + "\t" + str(start_of_words) + "\t" + str(end_of_words) + "\t" + ("".join(temp_phrase)).strip() + "\n")
        temp_phrase_storer =  temp_phrase_storer + copy.deepcopy(["".join(temp_phrase)])
        temp_phrase = []

      count_of_words_in_sentence += 1 

  f1.close()  
  phrase_storer =  phrase_storer + copy.deepcopy([temp_phrase_storer])
  print("done")

print("Here: ",len(phrase_storer))    

f.close()


155
13
['parsing as language modeling', 'we recast syntactic parsing as a language modeling problem and use recent advances in neural network language modeling to achieve a new state of the art for constituency penn treebank parsing - 93.8 f 1 on section 23 , using 2 - 21 as training , 24 as development , plus tri-training .', 'when trees are converted to stanford dependencies , uas and las are 95.9 % and 94.1 % .', 'in this paper we borrow from the approaches of both of these works and present a neural - net parse reranker that achieves very good results , 93.8 f 1 , with a comparatively simple architecture .', 'language modeling', 'in this paper , we build a parsing model based on the lstm - lm of .', 'the forget gate bias is initialized to be one and the rest of model parameters are sampled from u ( ? 0.05 , 0.05 ) .', 'dropout is applied to non-recurrent connections and gradients are clipped when their norm is bigger than 20 .', 'the learning rate is 0.25 0.85 max where is an epoch

#Evaluation Script for Phrases

This is the script provided by the organizers for phrase evaluation.
The first argument is the path to the folder containing true labels, second argument is the path to the folder containing predicted labels,
third argument is the path to the folder where the "scores.txt" will be stored.
###**IMPORTANT**

Please download the evaluation_for_phrases.py file and place it in /content folder and direct it appropriately.

In [ ]:
!python /content/drive/MyDrive/CS779_All_Subtasks/evaluation_for_phrases.py "/content/drive/MyDrive/test" "/content/" "/content/"